# Języki i Biblioteki Analizy Danych
## Laboratorium 6.: Generatory
#### mgr inż. Zbigniew Kaleta

* Generatorów używamy, aby oszczędzić pamięć (a także czas potrzebny na jej alokację).
* Zysk wydajności powstaje przez ominięcie potrzeby tworzenia tymczasowych struktur pośrednich w pamięci, gdy zamiast tego możemy przeiterować kolejno po elementach i finalnie zapisać tylko te, które są potrzebne.
* Generator to "funkcja po której można iterować", w każdej iteracji zwracająca kolejną wartość

### polecenie ``yield``

Funkcja zwracająca listę kolejnych elementów ciągu Fibonacciego nie przekraczających zadanej wartości.

In [ ]:
def fib(n=100):
    result = []
    a, b = 0, 1
    while b < n:
        result += [b]
        a, b = b, a + b
    return result

In [ ]:
fib()

To samo w postaci generatora

In [ ]:
def fib(n=100):
    a, b = 0, 1
    while b < n:
        yield b  # <- please note 'yield' in place of appending to list
        a, b = b, a + b

In [ ]:
fib()

In [ ]:
# typowy sposób użycia generatora
for i in fib():
    print(i)

In [ ]:
# a jeśli chcę mieć wszystkie wartości w liście...
list(fib())

In [ ]:
def f():
    yield 2
    yield 3
    return 4  # return powoduje rzucenie StopIteration z wiadomością 4; ten generator nie wygeneruje ani 4, ani 5
    yield 5

for i in f():
    print(i)

In [ ]:
def xrange(n):
    i = 0
    while i < n:
        yield i
        i += 1
        
list(xrange(10))

In [ ]:
def xrange(n):
    i = 0
    while i < n:
        j = yield i  # yield pozwala na komunikację dwukierunkową - "zwracam wartość i czekam na odpowiedź"
        if j is not None:
            i = j
        else:
            i += 1
        
g = xrange(10)
print(next(g))

In [ ]:
print(next(g))

In [ ]:
print(g.send(1))  # wysyłam odpowiedź do yielda i przekazuję sterowanie do generatora (tak jak w przypadku wywołania next)

Patrz: coroutines... ale potoki (pipeline) generatorów są lepsze

### Krótka historia range'a

In [ ]:
%%python2

print(range(10))
print(xrange(10))

In [ ]:
%%python3

print(range(10))
print(xrange(10))

### Generator expression

In [1]:
g = (i*i for i in range(10))  # takie same możliwości jak list comprehension, tylko generator

In [2]:
g

<generator object <genexpr> at 0x0000025C43E32810>

In [3]:
type(g)

generator

In [4]:
for i in g:
    print(i)
    if i>4:
        break

0
1
4
9


In [ ]:
for i in g:
    print(i)

In [ ]:
from time import perf_counter

In [5]:
start_time = perf_counter()
for i in [a**2 for a in range(20_000_000)]:
    break
print("DONE")
print("Time elapsed: {:.5f}s".format(perf_counter() - start_time))

NameError: name 'perf_counter' is not defined

In [ ]:
start_time = perf_counter()
for i in (a**2 for a in range(20_000_000)):
    break
print("DONE")
print("Time elapsed: {:.5f}s".format(perf_counter() - start_time))

Jedna z ważniejszych przewag generatora nad listą to możliwość przerwania generowania w dowolnym momencie. Nie marnujemy czasu na obliczenie wartości, które nie będą potrzebne.

Generator może również zawierać w sobie pętlę nieskończoną, a "obowiązek" jej przerwania będzie spoczywał na użytkowniku tego generatora.

Wynik testu z pełną iteracją po tej kolekcji (dla 2 mln elementów):

Lista: 0.50881s

Generator: 0.49612s

In [ ]:
g = (i*i for i in range(10) if i%2 for j in range(2))
list(g)

### Lista czy generator?

Jeżeli elementów do zwrócenia jest mało, to się nie zastanawiamy, tylko piszemy co nam wygodniej.

In [ ]:
def f():
    return [1, 2, 3]
    
def g():
    yield 1
    yield 2
    yield 3

Jeśli interesuje nas tylko jedna iteracja po elementach, to właściwie na jedno wychodzi.

In [ ]:
l = f()
for i in l:
    print(i)

In [ ]:
l = g()
for i in l:
    print(i)

Jeżeli potrzebujemy iterować wielokrotnie (albo tym bardziej potrzebujemy dostępu swobodnego), to lista jest nieodzowna...

In [ ]:
l = f()
for i in l:
    print(i)
for i in l:
    print(i)

In [ ]:
l = g()
for i in l:
    print(i)
for i in l:
    print(i)

... przy czym może ją sobie stworzyć kod wywołujący.

In [ ]:
l = list(g())
for i in l:
    print(i)
for i in l:
    print(i)

Dwukrotne wywołanie generatora będzie niewydajne, ale może się przydać, jeżeli pracujemy na sprzęcie z bardzo ograniczoną pamięcią, a nasz ciąg jest bardzo długi.

In [ ]:
l = g()
for i in l:
    print(i)
l = g()
for i in l:
    print(i)

### Funkcja czy generator?

In [ ]:
def f():
    return 1

print(f())

In [ ]:
def f():
    yield 1

print(f())

In [ ]:
print(next(f()))  # niewygodne w użyciu; generator nie zastąpi zwykłej funkcji

In [ ]:
def f():
    yield from [1, 2, 3]
    
print(f)
print(f())

### Chaining (pipelining) generatorów

Potok generatorów polega na tym, że jeden generator pobiera wartości z drugiego i zwraca je dalej, gdzie może czekać kolejny generator. Na początku potoku może być generator, ale równie dobrze może to być cokolwiek innego iterowalnego, jak lista, czy plik.

Poszczególne składowe potoku mogą zmniejszać liczbę elementów (np. odfiltrowywać liczby nieparzyste), zostawiać taką samą (np. podnosić liczby do kwadratu) lub zwiększać (np. rozbijać liczbę na poszczególne cyfry).

In [ ]:
def gen1():
    for i in range(100000):
        yield i
    
def filter1(gen):
    for i in gen:
        if i%2:
            yield i

def filter2(gen):
    for i in gen:
        if not i%11:
            yield i

for i in filter2(filter1(gen1())):
    print(i)

Gdyby każda z przedstawionych wyżej funkcji zwracała listę, to nadal by wszystko działało, tylko w każdym momencie potrzebowalibyśmy mieć dwie listy (lista zwrócona przez poprzedni element potoku i aktualnie tworzona). Przy dużych rozmiarach tych list zysk pamięciowy, a może również wydajnościowy, będzie zauważalny.

Lektura dodatkowa:
 - https://www.geeksforgeeks.org/generators-in-python/
 - https://realpython.com/introduction-to-python-generators
 - https://wiki.python.org/moin/Generators